In [ ]:
!gdown 1UzC3NCDj30j9Ba7i5lkMzWO5gFqSr0OJ
!unzip PMEmo2019.zip
!rm -rf ./__MACOSX
!rm -rf ./PMEmo2019.zip

!git clone https://github.com/rxng8/Music-Emotion-Recognition-Research.git

!mkdir ./Music-Emotion-Recognition-Research/data/PMEmo2019
!mv ./PMEmo2019 ./Music-Emotion-Recognition-Research/data/PMEmo2019

%cd ./Music-Emotion-Recognition-Research/

!sudo apt-get install libportaudio2
!apt install ffmpeg

!pip install -r ./requirements.txt

!python ./old/wav_converter.py "./data/PMEmo2019/PMEmo2019/chorus" "./data/PMEmo2019/PMEmo2019/chorus_wav" ffmpeg

!rm -rf ./configs/config.json

In [11]:
%%writefile ./configs/config.json
{
  "DEFAULT_FREQ": 44100,
  "DEFAULT_TIME": 40,
  "WINDOW_TIME": 5,
  "TRAIN_RATIO": 0.8,
  "BATCH_SIZE": 16,
  "FREQUENCY_LENGTH": 129,
  "N_CHANNEL": 1,
  "SPECTROGRAM_TIME_LENGTH": 15502,
  "SPECTROGRAM_HALF_SECOND_LENGTH": 171,
  "SPECTROGRAM_5_SECOND_LENGTH": 1721,
  "MFCCS_TIME_LENGTH": 3876,
  "LEARNING_RATE": 1e-4,
  "SOUND_EXTENSION": ".wav",
  "MIN_TIME_END_POINT": 15,
  "AUDIO_FOLDER": "./data/PMEmo2019/PMEmo2019/chorus_wav",
  "ANNOTATION_SONG_LEVEL": [
    "./data/PMEmo2019/PMEmo2019/annotations/static_annotations.csv",
    "./data/PMEmo2019/PMEmo2019/annotations/static_annotations_std.csv"
  ]
}

Writing ./configs/config.json


In [12]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
@Creator: Viet Dung Nguyen
@Date: April 14, 2022
@Credits: Viet Dung Nguyen
@Version: 0.0.1

Example notebook file
"""

import argparse
import os
import numpy as np
import librosa
import matplotlib.pyplot as plt

import tensorflow as tf
print(f"Tensorflow version: {tf.__version__}")
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)
from tensorflow.python.keras import layers as L
from tensorflow.python.keras.models import Model

from mer.mer.utils.const import get_config_from_json, setup_global_config

# Argument parsing
config_path = "./configs/config.json"
config = get_config_from_json(config_path)

##### Workaround to setup global config ############
setup_global_config(config, verbose=True)
from mer.mer.utils.const import GLOBAL_CONFIG
##### End of Workaround #####

# Because the generator and some classes are based on the
# GLOBAL_CONFIG, we have to import them after we set the config
from mer.mer.utils.utils import load_metadata, split_train_test, \
  preprocess_waveforms, get_spectrogram, plot_and_play, \
  pad_waveforms, plot_wave
from mer.mer.model import get_rnn_model,get_rnn_model_2, Simple_CRNN_3
from mer.mer.optimizer import get_Adam_optimizer, get_SGD_optimizer
from mer.mer.loss import simple_mae_loss, simple_mse_loss
from mer.mer.feature import load_wave_data, extract_spectrogram_features



Tensorflow version: 2.8.0
Num GPUs Available:  0
Munch({'DEFAULT_FREQ': 44100, 'DEFAULT_TIME': 40, 'WINDOW_TIME': 5, 'TRAIN_RATIO': 0.8, 'BATCH_SIZE': 16, 'FREQUENCY_LENGTH': 129, 'N_CHANNEL': 1, 'SPECTROGRAM_TIME_LENGTH': 15502, 'SPECTROGRAM_HALF_SECOND_LENGTH': 171, 'SPECTROGRAM_5_SECOND_LENGTH': 1721, 'MFCCS_TIME_LENGTH': 3876, 'LEARNING_RATE': 0.0001, 'SOUND_EXTENSION': '.wav', 'MIN_TIME_END_POINT': 15, 'AUDIO_FOLDER': './data/PMEmo2019/PMEmo2019/chorus_wav', 'ANNOTATION_SONG_LEVEL': ['./data/PMEmo2019/PMEmo2019/annotations/static_annotations.csv', './data/PMEmo2019/PMEmo2019/annotations/static_annotations_std.csv'], 'WAVE_ARRAY_LENGTH': 1764000, 'WINDOW_SIZE': 220500})


In [13]:
# %%%
filenames = tf.io.gfile.glob(str(GLOBAL_CONFIG.AUDIO_FOLDER) + '/*')
# Process with average annotation per song. 

df = load_metadata(GLOBAL_CONFIG.ANNOTATION_SONG_LEVEL)

# Smaller set of data
# df = df[:64]

df


,musicId,Arousal(mean),Valence(mean),Arousal(std),Valence(std)
0,1,0.4000,0.5750,0.156125,0.160078
1,4,0.2625,0.2875,0.152582,0.158607
2,5,0.1500,0.2000,0.175000,0.203101
3,6,0.5125,0.3500,0.171847,0.200000
4,7,0.7000,0.7250,0.139194,0.122474
...,...,...,...,...,...
762,993,0.8625,0.7625,0.103833,0.152582
763,996,0.8750,0.5625,0.111803,0.245268
764,997,0.7125,0.6625,0.177218,0.112500
765,999,0.8750,0.7750,0.096825,0.122474


In [14]:

# %%

train_df, test_df = split_train_test(df, GLOBAL_CONFIG.TRAIN_RATIO)

# %%

# song_path = "../data/PMEmo/PMEmo2019/PMEmo2019/chorus_wav/6.wav"

# lib_wave, sr = librosa.load(song_path, GLOBAL_CONFIG.DEFAULT_FREQ)
# lib_wave = tf.convert_to_tensor(lib_wave)[..., tf.newaxis]

# plot_wave(lib_wave, second_length=40)


# audio_file = tf.io.read_file(song_path)

# waveforms, sample_rate = tf.audio.decode_wav(contents=audio_file)


# def plot_wave_4(waveforms, second_id = 0, second_length = 10, channel = 0):
#   from_id = int(44100 * second_id)
#   to_id = min(int(44100 * (second_id + second_length)), waveforms.shape[0])

#   fig, axes = plt.subplots(1, figsize=(12, 4))
#   timescale = np.arange(to_id - from_id)
#   axes.plot(timescale, waveforms[from_id:to_id, channel].numpy())
#   axes.set_title('Waveform')
#   axes.set_xlim([0, int(44100 * second_length)])
#   plt.show()

# plot_wave_4(waveforms, second_length=40)




In [15]:

# %%

def train_datagen():
  """ Predicting valence mean and arousal mean
  """
  pointer = 0
  while True:
    # Reset pointer
    if pointer >= len(train_df):
      pointer = 0

    row = train_df.loc[pointer]
    song_id = row["musicId"]
    arousal_mean = float(row["Arousal(mean)"])
    valence_mean = float(row["Valence(mean)"])
    
    # TODO: HOw are we gonna integrate valence and arousal std?
    arousal_std = float(row["Arousal(std)"])
    valence_std = float(row["Valence(std)"])
    
    label = tf.convert_to_tensor([valence_mean, arousal_mean, valence_std, arousal_std], dtype=tf.float32)
    song_path = os.path.join(GLOBAL_CONFIG.AUDIO_FOLDER, str(int(song_id)) + GLOBAL_CONFIG.SOUND_EXTENSION)
    
    waveforms = load_wave_data(song_path)
    
    spectrogram_features = extract_spectrogram_features(waveforms)
    
    # Preprocessed and normalize waveforms in the end
    waveforms = preprocess_waveforms(waveforms)

    # Update pointer
    pointer += 1
    yield (waveforms, spectrogram_features, label)

def test_datagen():
  """ Predicting valence mean and arousal mean
  """
  pointer = 0
  while True:
    # Reset pointer
    if pointer >= len(test_df):
      pointer = 0

    row = test_df.loc[pointer]
    song_id = row["musicId"]
    valence_mean = float(row["Valence(mean)"])
    arousal_mean = float(row["Arousal(mean)"])
    arousal_std = float(row["Arousal(std)"])
    valence_std = float(row["Valence(std)"])
    label = tf.convert_to_tensor([valence_mean, arousal_mean, valence_std, arousal_std], dtype=tf.float32)
    song_path = os.path.join(GLOBAL_CONFIG.AUDIO_FOLDER, str(int(song_id)) + GLOBAL_CONFIG.SOUND_EXTENSION)

    waveforms = load_wave_data(song_path)
    
    spectrogram_features = extract_spectrogram_features(waveforms)
    
    # Preprocessed and normalize waveforms in the end
    waveforms = preprocess_waveforms(waveforms)

    pointer += 1
    yield (waveforms, spectrogram_features, label)

train_dataset = tf.data.Dataset.from_generator(
  train_datagen,
  output_signature=(
    tf.TensorSpec(shape=(GLOBAL_CONFIG.WAVE_ARRAY_LENGTH, GLOBAL_CONFIG.N_CHANNEL), dtype=tf.float32),
    tf.TensorSpec(shape=(GLOBAL_CONFIG.SPECTROGRAM_TIME_LENGTH, GLOBAL_CONFIG.FREQUENCY_LENGTH, GLOBAL_CONFIG.N_CHANNEL), dtype=tf.float32),
    tf.TensorSpec(shape=(4), dtype=tf.float32)
  )
)
train_batch_dataset = train_dataset.batch(GLOBAL_CONFIG.BATCH_SIZE)
# train_batch_dataset = train_batch_dataset.cache().prefetch(tf.data.AUTOTUNE) # OOM error
train_batch_iter = iter(train_batch_dataset)

test_dataset = tf.data.Dataset.from_generator(
  test_datagen,
  output_signature=(
    tf.TensorSpec(shape=(GLOBAL_CONFIG.WAVE_ARRAY_LENGTH, GLOBAL_CONFIG.N_CHANNEL), dtype=tf.float32),
    tf.TensorSpec(shape=(GLOBAL_CONFIG.SPECTROGRAM_TIME_LENGTH, GLOBAL_CONFIG.FREQUENCY_LENGTH, GLOBAL_CONFIG.N_CHANNEL), dtype=tf.float32),
    tf.TensorSpec(shape=(4), dtype=tf.float32)
  )
)
test_batch_dataset = test_dataset.batch(GLOBAL_CONFIG.BATCH_SIZE)
# test_batch_dataset = test_batch_dataset.cache().prefetch(tf.data.AUTOTUNE) # OOM error
test_batch_iter = iter(test_batch_dataset)



In [16]:
# %%
# test data gen

in_wave, in_spec, out = next(test_batch_iter)
print(in_wave.shape)
print(in_spec.shape)
print(out.shape)

# plot_wave(in_wave[0], second_id = 0, second_length = 40, channel = 0)
# plot_and_play(in_wave[5], second_id = 0, second_length = 40, channel = 0)





(16, 1764000, 1)
(16, 15502, 129, 1)
(16, 4)


In [17]:
# %%

# model = get_rnn_model()
# model.summary()
# model_name = "rnn_1"
# sample_input = tf.ones(shape=(BATCH_SIZE, SPECTROGRAM_TIME_LENGTH, FREQUENCY_LENGTH, 2))
# with tf.device("/CPU:0"):
#   sample_output = model(sample_input, training=False)
# print(sample_output)

# model = get_rnn_model_2(input_shape=in_wave.shape[1:])
# model.summary()
# model_name = "rnn_2"

model = Simple_CRNN_3(input_shape=in_spec.shape[1:])
model.summary()
model_name = "crnn_3"



Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 15502, 129, 1)]   0         
_________________________________________________________________
permute (Permute)            (None, 129, 15502, 1)     0         
_________________________________________________________________
tf.convert_to_tensor (TFOpLa (None, 129, 15502, 1)     0         
_________________________________________________________________
tf.image.resize (TFOpLambda) (None, 129, 2048, 1)      0         
_________________________________________________________________
tf.cast (TFOpLambda)         (None, 129, 2048, 1)      0         
_________________________________________________________________
conv2d (Conv2D)              (None, 125, 2044, 64)     1664      
_________________________________________________________________
re_lu (ReLU)                 (None, 125, 2044, 64)     0     

In [18]:
# %%


history_path = f"./history/{model_name}.npy"
weights_path = f"./models/{model_name}/checkpoint"

# optimizer = get_SGD_optimizer()
optimizer = get_Adam_optimizer()



In [ ]:
# %%

from mer.mer.trainer import Trainer

trainer = Trainer(model,
  train_batch_iter,
  test_batch_iter,
  optimizer,
  simple_mse_loss,
  epochs=10,
  steps_per_epoch=45, # // 64 // 16 // //////     724 // 16 = 45
  valid_step=5,
  history_path=history_path,
  weights_path=weights_path,
  save_history=True)

# About 50 epochs with each epoch step 100 will cover the whole training dataset!
history = trainer.train()




In [ ]:
# %%

from mer.mer.utils.utils import plot_history

plot_history(history_path)



In [ ]:
# %%

def evaluate(df_pointer, model, loss_func, play=False):
  row = test_df.loc[df_pointer]
  song_id = row["musicId"]
  arousal_mean = float(row["Arousal(mean)"])
  valence_mean = float(row["Valence(mean)"])
  arousal_std = float(row["Arousal(std)"])
  valence_std = float(row["Valence(std)"])
  label = tf.convert_to_tensor([valence_mean, arousal_mean, valence_std, arousal_std], dtype=tf.float32)
  print(f"Label: Valence: {valence_mean}, Arousal: {arousal_mean}")
  song_path = os.path.join(GLOBAL_CONFIG.AUDIO_FOLDER, str(int(song_id)) + GLOBAL_CONFIG.SOUND_EXTENSION)
  waveforms = load_wave_data(song_path)
  spectrograms = extract_spectrogram_features(waveforms)[tf.newaxis, ...]
  waveforms = preprocess_waveforms(waveforms)

  ## Eval
  y_pred = model(spectrograms, training=False)[0]

  print(y_pred.shape)

  print(f"Predicted y_pred value: Valence: {y_pred[0]}, Arousal: {y_pred[1]}")

  loss = loss_func(label[tf.newaxis, ...], y_pred)
  print(f"Loss: {loss}")

  if play:
    plot_and_play(waveforms, 0, 40, 0)

i = 0




In [ ]:
# %%

# model.load_weights(weights_path)

# %%

i += 1
evaluate(i, model, simple_mse_loss, play=True)